In [2]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
from openai import OpenAI
import os
import json
import random
import time

In [3]:
# load data
df = pd.read_excel("data/intermediate/openai_results.xlsx")

In [4]:
df

,Unnamed: 0,topic,short_sumary,key_aspect,key_word,positve,rating,assagekraft,sarcasm_rating,value,document,column_name
0,0,Arbeitsatmosphäre,"Herzlich, freundlich, hilfsbereit und positiv",Atmosphäre,lebendig,True,9,8,False,"Herzlich, freundlich, hilfsbereit und voralle...",1,working_atmosphere_comment
1,1,Mentoring,"Super Mentorin, die persönliche Weiterentwickl...",Mentoring,Unterstützung,True,9,8,False,"Ich habe einer super Mentorin , die mir die Mö...",1,leadership_behavior_comment
2,2,Arbeitsatmosphäre,Sehr hilfsbereit und auch neben dem Job trinkt...,Teamwork,gemeinschaftlich,True,8,7,False,Sehr hilfsbereit und auch neben dem Job trinkt...,1,team_spirit_comment
3,3,Beratung,Arbeit mit Menschen,Vielseitigkeit,Spaß,True,8,7,False,Bei der Beratung steht der Mensch im Mittelpun...,1,interesting_tasks_comment
4,4,Freundlichkeit,Die Mitarbeiter waren sehr freundlich und hilf...,Mitarbeiter,freundlich,True,8,9,False,"Freundlich, konstruktiv und höflich.",1,communication_comment
...,...,...,...,...,...,...,...,...,...,...,...,...
1164,1164,Kollegen und Zusammenhalt,Klasse Zusammenhalt,Zusammenarbeit,Kollegen,True,9,8,False,Die Kollegen und der Zusammenhalt ist klasse!,200,team_spirit_comment
1165,1165,Arbeitsinhalt,Die Arbeit macht Spaß und ist vielseitig,workload,hoch,True,8,7,False,Die Arbeit macht Spaß und ist vielseitig...es ...,200,interesting_tasks_comment
1166,1166,Transparenz,Manchmal fehlt die Transparenz,Informationsweitergabe,spät,False,7,5,False,"Allgemein gut, nur manchmal fehlt die Transpar...",200,communication_comment
1167,1167,Arbeitsumgebung,"Systeme hängen ab und an, sonst alles top! Küc...",Systemstabilität,ab und an,False,8,7,False,"Systeme hängen ab und an, sonst alles top! Küc...",200,working_conditions_comment


In [4]:
# Creating a list of tuples (index, original row number, column name, value)
records = []
counter = 1
for index, row in df_raw.iterrows():
    for col in df_raw.columns:
        records.append((counter, index + 1, col, row[col]))  # index + 1 to make it 1-based
        counter += 1

# Convert the list of tuples to a DataFrame
df = pd.DataFrame(records, columns=['Index', 'Document', 'ColumnName', 'Value'])
df = df.drop(columns=['Index'])

In [5]:

# set function output format
class StepByStepAIResponse(BaseModel):
    topic: str
    short_sumary: str
    key_aspect: str
    key_word: str
    positve: bool
    rating: int
    assagekraft: int
    sarcasm_rating: bool
    
# set openai api key
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [10]:
# Define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        num_retries = 0
        delay = initial_delay

        while True:
            try:
                return func(*args, **kwargs)

            except e:
                num_retries += 1

                if num_retries > max_retries:
                    raise Exception(f"Maximum number of retries ({max_retries}) exceeded.")

                delay *= exponential_base * (1 + jitter * random.random())

                time.sleep(delay)

            except Exception as e:
                raise e

    return wrapper

In [12]:

@retry_with_exponential_backoff
def call_open_ai(client: OpenAI, statement_df: pd.Series) -> dict:
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": statement_df['Value'],
            }
        ],
        functions=[
            {
            "name": "beschreibe_jobportal_bewertung",
            "description": """Beschreibe eine Jobportal-Bewertung mit einer übergeordneten Topic, 
            einer extrem kurzen Zusammenfassen, dem key-aspect, dem key-word, 
            einem positivem or negativem Sentiment, einer geschätzen Abeitgeber-Attraktivitäts Bewertung 0-10, 
            einer geschätzen Aussagekraft der Bewertung 0-10 (sei kritisch!), einer Sarkasmus Einschätzung 0-10""",
            "parameters": StepByStepAIResponse.model_json_schema()
            }
        ],
        function_call={"name": "beschreibe_jobportal_bewertung"},
        temperature = 0.2,
        timeout=30,
    )

    completion_value = completion.choices[0].message.function_call.arguments
    completion_dict = json.loads(completion_value)
    completion_dict['value'] = statement_df['Value']
    completion_dict['document'] = statement_df['Document']
    completion_dict['column_name'] = statement_df['ColumnName']
    
    return completion_dict
    


In [13]:
results = []
for index, row in df.iterrows():
    
    # ignore missing values
    if row['Value'] is np.nan:
        continue
    
    # call open ai
    result = call_open_ai(client, row)
    results.append(result)
    
    print(f"{index}: {result['key_aspect']}")
    
    if index % 20 == 0:
        result_df = pd.DataFrame(results)
        result_df.to_excel("data/openai_results.xlsx")

0: Atmosphäre
1: Mentoring
2: Teamwork
3: Vielseitigkeit
4: Mitarbeiter
5: Center
6: Arbeitszeit
7: Arbeitskollegen und Chefs
8: Soziales Engagement
9: Teamzusammenhalt
10: Menschlichkeit
12: Wetterbedingungen
13: Belastung
14: Arbeitsatmosphäre
15: Unternehmenskultur
16: Einzigartigkeit
17: persönliche Lernkurve
18: Transparenz
19: Ausstattung
20: Struktur
21: Nettigkeit, Hilfsbereitschaft, Offenheit, Sympathie, positive Herangehensweise
22: Unterstützung und Förderung
25: Kultur und Hierarchien
28: Kollegiales Miteinander
32: Kommunikation
33: Arbeitsumgebung
34: Zur Verfügung gestellte Ressourcen
35: Struktur
36: Nettigkeit der Filialleitung
37: Nettigkeit und Interesse
38: Kontakt zu Kunden
39: Benutzerfreundlichkeit
40: Kassensystem, Pausenraum, Sauberkeit
41: Gehalt
42: Bürogestaltung
43: Umgang mit Mitarbeitern
47: Arbeitsumgebung
48: Wohlbefinden der Mitarbeiter
49: Freundlichkeit und Hilfsbereitschaft
50: Kommunikation
51: Zusammenhalt
54: Einrichtung
55: Mitarbeiterwohlbefind

In [15]:
result_df = pd.DataFrame(results)
result_df.to_excel("data/openai_results.xlsx")

In [18]:
str(result_df['key_aspect'].to_list())

"['Atmosphäre', 'Mentoring', 'Teamwork', 'Vielseitigkeit', 'Mitarbeiter', 'Center', 'Arbeitszeit', 'Arbeitskollegen und Chefs', 'Soziales Engagement', 'Teamzusammenhalt', 'Menschlichkeit', 'Wetterbedingungen', 'Belastung', 'Arbeitsatmosphäre', 'Unternehmenskultur', 'Einzigartigkeit', 'persönliche Lernkurve', 'Transparenz', 'Ausstattung', 'Struktur', 'Nettigkeit, Hilfsbereitschaft, Offenheit, Sympathie, positive Herangehensweise', 'Unterstützung und Förderung', 'Kultur und Hierarchien', 'Kollegiales Miteinander', 'Kommunikation', 'Arbeitsumgebung', 'Zur Verfügung gestellte Ressourcen', 'Struktur', 'Nettigkeit der Filialleitung', 'Nettigkeit und Interesse', 'Kontakt zu Kunden', 'Benutzerfreundlichkeit', 'Kassensystem, Pausenraum, Sauberkeit', 'Gehalt', 'Bürogestaltung', 'Umgang mit Mitarbeitern', 'Arbeitsumgebung', 'Wohlbefinden der Mitarbeiter', 'Freundlichkeit und Hilfsbereitschaft', 'Kommunikation', 'Zusammenhalt', 'Einrichtung', 'Mitarbeiterwohlbefinden', 'Atmosphäre', 'Kommunikation